# Plotting particle distributions with holoviews

In [ ]:
import numpy as np

# from netCDF4 import Dataset
import xarray as xr
import holoviews as hv
from postladim import ParticleFile

hv.extension("bokeh")

### Background map

Make a background bathymetric map.
A simple land representation is given by colouring the land cells in the
ROMS file. Take the logarithm of the bathymetry to enhance topographic details
in the shallow North Sea.

In [ ]:
# Read bathymetry and land mask
with xr.open_dataset("../data/ocean_avg_0014.nc") as ds:
    H = ds.h  # Bottom topography
    M = ds.mask_rho  # Land mask
jmax, imax = M.shape
H = H.where(M > 0)  # Mask out land
M = M.where(M < 1)  # Mask out sea

In [ ]:
# Land image
land = hv.Image(data=M, kdims=["xi_rho", "eta_rho"], group="Land")

# Bathymetry image
# Take logarithm to show details in the shallow North Sea
topo = hv.Image(data=np.log10(H), kdims=["xi_rho", "eta_rho"], group="Topo")

background = topo * land

### Particle plot function

Open the particle file and make a function to make a
Scatter element of the particle distribution at a given time step.

In [ ]:
pf = ParticleFile("out.nc")


def pplot(timestep):
    """Scatter plot of particle distibution at a given time step"""
    X, Y = pf.position(timestep)
    return background * hv.Points((X, Y))

### Still images

Set a green colour on land and use shades of blue at sea. Show initial
and final particle distributions.

In [ ]:
%%opts Image.Topo (cmap='blues' alpha=0.7) 
%%opts Image.Land (cmap=['#509030'])    
%%opts Points (color='red')

pplot(0) + pplot(pf.num_times-1)  # Initial and final particle distributions

### Dynamic map

Make a DynamicMap of all the particle distributions. Choose time step with a slider.

In [ ]:
%%output size=150
%%opts Points (color='red')

dmap = hv.DynamicMap(pplot, kdims=['timestep'])
dmap.redim.range(timestep=(0, pf.num_times-1))

In [ ]:
# Read in a presaved land contour
coast_file = "coast_grid.npy"
with open(coast_file, "rb") as f:
    Xcoast = np.load(f)
    Ycoast = np.load(f)

In [ ]:
coast = hv.Polygons({"x": Xcoast, "y": Ycoast})


# background2 = topo * coast
# topo2 = hv.Image(data=np.log10(H[10:-10,10:-10]), kdims=["xi_rho", "eta_rho"])
def pplot2(timestep):
    """Scatter plot of particle distibution at a given time step"""
    X, Y = pf.position(timestep)
    return topo * coast * hv.Points((X, Y))

In [ ]:
%%output size=150
%%opts Points (color='red')
%%opts Polygons (color="lightgrey")
dmap = hv.DynamicMap(pplot2, kdims=['timestep'])
dmap.redim.range(timestep=(0, pf.num_times-1))

In [ ]:
with xr.open_dataset("../data/ocean_avg_0014.nc") as ds:
    H = ds.h  # Bottom topography
    M = ds.mask_rho  # Land mask
    lon = ds.lon_rho.values
    lat = ds.lat_rho.values
jmax, imax = M.shape
X, Y = np.meshgrid(np.arange(imax), np.arange(jmax))

lo, la = 0, 60
import scipy

np.column_stack((lon.ravel(), lat.ravel()))

scipy.interpolate.griddata(
    np.column_stack((lon.ravel(), lat.ravel())), Y.ravel(), [lo, la]
)

In [ ]:
?scipy.interpolate.griddata